In [ ]:
! pip install ultralytics opencv-python pytesseract
! apt-get install -y tesseract-ocr

# Python용 pytesseract 라이브러리 설치
! pip install pytesseract
! pip install opencv-python-headless

! apt-get install -y tesseract-ocr-kor

!pip install opencv-python
!pip install matplotlib

# yolo v5 clone
! git clone https://github.com/ultralytics/yolov5
! pip install -r /content/yolov5/requirements.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-kor is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
# 데이터셋 준비
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="prCkfgIyJ6xrZaI1Q5eG")
project = rf.workspace("school-b6rj0").project("-dpo7a")
version = project.version(1)
dataset = version.download("yolov5")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to 차량번호판-최종-1 in yolov5pytorch:: 100%|██████████| 2910/2910 [00:00<00:00, 5316.05it/s]


In [ ]:
# train
! python3 /content/drive/MyDrive/오픈소스SW프로그래밍/project/yolov5/train.py --img 640 --batch 16 --epochs 30 --data /content/drive/MyDrive/오픈소스SW프로그래밍/project/yolov5/data.yaml --weights /content/drive/MyDrive/오픈소스SW프로그래밍/project/yolov5s.pt

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-24 10:21:40.660762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 10:21:40.681543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 10:21:40.687440: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [ ]:
# 번호판 탐지

import cv2
import torch
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
import os

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# YOLOv5 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/오픈소스SW프로그래밍/project/yolov5/runs/train/exp6/weights/best.pt') # epoch 30

# 이미지 경로
image_path = '/content/drive/MyDrive/[오픈소스SW]3팀_데이터셋/강한성/jpg_ver/ (59).jpg'

# 이미지 불러오기
img = cv2.imread(image_path)

# 이미지 확인
if img is None:
    print("Image not found or invalid path.")
else:
    print(f"Image loaded successfully. Shape: {img.shape}")

# YOLOv5로 객체 탐지
results = model(img)

detections = results.xyxy[0]  # 탐지된 객체의 좌표
for i, (*box, conf, cls) in enumerate(detections):  # 좌표, 신뢰도, 클래스
    x1, y1, x2, y2 = map(int, box)  # 바운딩 박스 좌표
    cropped_plate = img[y1:y2, x1:x2]  # 차량번호판 영역 자르기

output_path = os.path.join('.', f'plate.jpg')
cv2.imwrite(output_path, cropped_plate)
print(f"Saved cropped plate image to: {output_path}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-24 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Image loaded successfully. Shape: (4032, 3024, 3)
Saved cropped plate image to: ./plate.jpg


`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.


In [ ]:
# 전처리 과정
# ! 전처리하지 않은 사진에서 정확도가 더 높아 주석처리함

'''import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract

# 이미지 읽기
img = cv2.imread('/content/plate.jpg', cv2.IMREAD_COLOR)
copy_img = img.copy()


# 전처리: 그레이스케일 변환
gray = cv2.cvtColor(copy_img, cv2.COLOR_BGR2GRAY)

# 전처리: 블러링 및 엣지 검출
blur = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(blur,50,100)

# 탐지된 번호판 이미지를 저장
output_path = os.path.join('.', f'canny.jpg')
cv2.imwrite(output_path, canny)
print(f"Saved cropped plate image to: {output_path}")'''

'import cv2\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport pytesseract\n\n# 이미지 읽기\nimg = cv2.imread(\'/content/plate.jpg\', cv2.IMREAD_COLOR)\ncopy_img = img.copy()\n\n\n# 전처리: 그레이스케일 변환\ngray = cv2.cvtColor(copy_img, cv2.COLOR_BGR2GRAY)\n\n# 전처리: 블러링 및 엣지 검출\nblur = cv2.GaussianBlur(gray, (3, 3), 0)\ncanny = cv2.Canny(blur,50,100)\n\n# 탐지된 번호판 이미지를 저장\noutput_path = os.path.join(\'.\', f\'canny.jpg\')\ncv2.imwrite(output_path, canny)\nprint(f"Saved cropped plate image to: {output_path}")'

In [ ]:
# 번호판 텍스트 추출

config = ('--oem 3 --psm 6')
text = pytesseract.image_to_string('/content/plate.jpg', lang='kor', config=config)
print('text : ', text)

text :  [06도 49589

